# 4章 4.2 EGM を用いた TI の準コード

- $\delta < 1$
- $A_t = \{A^{good}, A^{bad}\}$

### モデルの説明

- CRRA型効用関数: $u(c) = \frac{c^{1-\gamma}}{1-\gamma}$
- 生産関数: $f(k; A) = Ak^\alpha$
  - 技術水準 $A$ は良い状態の時と悪い状態の時があるとする $\{A^{good}, A^{bad}\}$
  - 技術水準の遷移確率: $P_A = \begin{bmatrix} P_{gg} & P_{gb} \\ P_{bg} & P_{bb} \end{bmatrix}$
- 減耗した資本を含む富の関数: $m = \tilde{f}(k; A, \delta)$ = $f(k; A) + (1 - \delta)k$

In [61]:
# パッケージのインポート

import numpy as np
import numpy.typing as npt
from numpy.polynomial.chebyshev import Chebyshev
import matplotlib.pyplot as plt
import math

### 1. パラメータをカリブレーション

In [62]:
# パラメータをカリブレーション
beta = 0.96
gamma = 1.0
alpha = 0.4 
delta = 0.025

### 2. 収束の基準を定義

In [63]:
# 収束の基準を定義
epsilon = 1e-5

### 3. 制御変数のグリッドを生成

- $\{ k^\prime \}_{i = 1}^N, \;\; N = 21$
- $k_1^\prime = 0, k_{21}^\prime = 0.5$

In [ ]:
# 制御変数のグリッドを生成
nk = 20
Kprime_grid = np.linspace(0.025, 0.5, nk)
print(Kprime_grid)

### 4. 技術水準の状態$A \in \{A^{good}, A^{bad}\}$を定義

In [65]:
# 技術水準の状態を定義
na = 2
A_grid = np.zeros(na)
A_grid = np.array([1.01, 0.99])

### 5. $\{k_i^\prime\}^{N_k}_{i = 1}$と$\{A_k\}^{N_A}_{k=1}$から$m^\prime$のグリッド（二次元）を計算する

In [ ]:
# m'のグリッドを生成
def f_tilde(k: float, A: float) -> float:
    return (k**alpha) * A + (1 - delta) * k

Mprime_matrix = np.zeros((nk, na))
for i in range(nk):
    for j in range(na):
        Mprime_matrix[i, j] = f_tilde(Kprime_grid[i], A_grid[j])

print(Mprime_matrix)

### 5. $N_A \times N_A$の遷移確率行列を手置き定義 

In [67]:
# 遷移確率行列を定義
P = np.array([[0.875, 0.125], [0.125, 0.875]])

### 6. 初期値として政策関数を当て推量 $c_{i, j} = h^{(0)}(m^\prime_{i,j})$

In [ ]:
# 6. 初期値として政策関数を当て推量，富の増加関数とする
h_old_matrix = np.zeros((nk, na))
h_old_matrix = Mprime_matrix * 0.5
np.set_printoptions(precision=3)
print(h_old_matrix)

### 7. 次のステップを収束するまで繰り返す(繰り返し記号：n)
   1. 古い政策関数 $h^{(n-1)}(k_i^\prime)$ を所与として，$\{k^\prime_i\}_{i=1}^{N_k} \times A_j \in \{A^{good}, A^{bad}\}$ の各グリッドについて
   	
      - $c_{i, j} = u^{\prime -1} ( \beta \sum_{k=1}^{N_A} \{ P_{j,k} u^\prime(h^{(n-1)}(k^\prime_i, A_k)) \times \tilde{f}^\prime (k^\prime_i, A_k)\} )$
      - $c_{i,j} + k^\prime_i = \tilde{f}(k_i, A_j) \Rightarrow k_i = \tilde{f}^{-1} (c_{i,j} + k^\prime_i)$

In [69]:
# ステップ7の準備（ndarrayが前提）
def u_prime(c: float) -> float:
    return c**(-gamma)
def u_prime_inv(mu: float) -> float:
    return mu**(-1/gamma)
def f_tilde_prime(k: float, A: float) -> float:
    return alpha * A * (k**(alpha - 1)) + (1 - delta)

def interpolate_cheb(m_matrix,A_list,c_matrix):
    """チェビシェフ補間を行う関数を返す関数

    Args:
        m: 計算された m の行列 m[i, j]
        A (npt.NDArray): 説明変数 A の配列（離散化された状態の長さ分）
        c (npt.NDArray): 目的変数 c の配列
    """
    def interp_func(m,a):
        # A_list から aと一致する要素の番号を得る
        A_idx = np.where(A_list == a)[0][0]
        # m_matrixから k_matrix[i, z_idx]の配列を得る
        m_list = m_matrix[:, A_idx]
        c_list = c_matrix[:, A_idx]
        cheb_fit = Chebyshev.fit(m_list, c_list, deg=15)
        
        return cheb_fit(m)
    
    return interp_func

In [ ]:
diff = epsilon + 1
loop = 0
while diff > epsilon:
    loop += 1
    Gamma = np.zeros((nk, na)) # オイラー方程式の右辺
    C_matrix = np.zeros((nk, na))
    M_matrix = np.zeros((nk, na))
    
    for i in range(nk):
        for j in range(na):
            for k in range(na):
                Gamma[i, j] += beta * P[j, k] * u_prime(h_old_matrix[i, k]) * f_tilde_prime(Kprime_grid[i], A_grid[k])
            C_matrix[i, j] = u_prime_inv(Gamma[i, j])
            M_matrix[i, j] = C_matrix[i,j] + Kprime_grid[i]


    h_new_func = interpolate_cheb(M_matrix, A_grid, C_matrix)
    np_h_new_func = np.frompyfunc(h_new_func, 2, 1)

    # # 収束の確認 
    A_matrix = np.array(list(A_grid) * nk).reshape(nk, na)
    h_new_matrix = np_h_new_func(Mprime_matrix, A_matrix) #h_new_matrixには、次期の政策関数が入流べき！??
    diff = np.max(np.abs(h_new_matrix - h_old_matrix))
    
    h_old_matrix = np.copy(h_new_matrix)
    if loop % 50 == 0:
        print(f"loop: {loop}, diff: {diff}")

print(h_old_matrix)

### 政策関数を 3次元プロット

In [ ]:
# np_h_new_func を使って三次元プロット
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(M_matrix, A_matrix, C_matrix, cmap='viridis')
ax.set_xlabel('M')
ax.set_ylabel('A')
ax.set_zlabel('C')
plt.show()
print(loop)
print(diff)

### z の値を固定した政策関数の二次元プロット

In [ ]:
for i in range(na):
    M = M_matrix[:, i]
    C = C_matrix[:, i]
    plt.plot(M, C, label=f'A = {A_grid[i]}')
plt.xlabel('M')
plt.ylabel('C')
plt.legend()
plt.show()

### Kprime と K のプロット

In [ ]:
# K と K' の関係をプロット
for i in range(na):
    K = np.copy(Kprime_grid)
    for j  in range(len(K)):
        M[j] = f_tilde(K[j], A_grid[i])
        C[j] = h_new_func(M[j], A_grid[i])
    Kprime = M - C
    plt.plot(K, Kprime, label=f'A = {A_grid[i]}')
plt.xlabel('K')
plt.ylabel('Kprime')
plt.legend()
plt.show()

### $Z_t$ の流列を生成し、$K, C$の流列をシミュレーション

In [ ]:
# 確率的シミュレーション
T = 100
# 0-100まで
T_series = np.linspace(0, T, T)
K_path = np.zeros(T)
Z_path = np.zeros(T)
C_path = np.zeros(T)
K_path[0] = 1.0

# Z_gridから Z_pathの値をランダムに生成
for t in range(T):
  Z_path[t] = Z_grid[np.random.choice(nz)]

for t in range(1, T):
  C_path[t-1] = h_new_func(f_tilde(K_path[t-1],Z_path[t-1]), Z_path[t-1])
  K_path[t] = f_tilde(K_path[t-1], Z_path[t-1]) - C_path[t]

# K_path, C_path, Z_path を T_series を横軸に3つ縦に並べてプロット
# グラフの描画領域
fig = plt.figure()
# サブプロットの追加
ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)
# プロット
ax1.plot(T_series, K_path, label='K')
ax2.plot(T_series, C_path, label='C')
ax3.plot(T_series, Z_path, label='Z')
# ラベル
ax1.set_ylabel('K')
ax2.set_ylabel('C')
ax3.set_ylabel('Z')
# タイトル
ax1.set_title('K_path')
ax2.set_title('C_path')
ax3.set_title('Z_path')
fig.tight_layout()
plt.xlabel('T')
plt.show()

### リセット

In [60]:
%reset -f